# CERT Framework: Agentic Evaluation with LangChain

This notebook demonstrates how to properly evaluate agentic AI pipelines using the CERT SDK with LangChain.

## Key Concepts

**The Context Requirement**: In agentic evaluation, tool outputs serve as the *implicit context* that enables full reliability metrics. Without context, evaluation degrades to Generation Mode with only weak quality signals.

| Evaluation Mode | Context Source | Available Metrics |
|-----------------|----------------|-------------------|
| RAG | Explicit context | Semantic similarity, NLI, Grounding, **SGI** |
| Agentic | Tool outputs (auto-extracted) | Same as RAG + tool integration |
| Generation | None | Instruction adherence, self-consistency, format |

**The CERT SDK automatically extracts context from tool outputs in agentic mode**, enabling computation of SGI (Source-Grounding Index) and other reliability metrics.

## Setup

In [ ]:
# Install dependencies
!pip install langchain langchain-openai langchain-core -q

# Install CERT SDK (in production, use: pip install cert-sdk[langchain])
# For now, install from local
!pip install -e ./cert-sdk-main[langchain] -q

In [ ]:
import os
import json
import time
from typing import Optional, Type

# Set API keys
os.environ["OPENAI_API_KEY"] = "your-openai-key-here"
# os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-key-here"

CERT_API_KEY = "cert_demo_key"  # Get from dashboard.cert-framework.com

## 1. Define Tools

We'll create a simple agent with three tools:
- **Weather API**: Get current weather
- **Calculator**: Perform calculations
- **Search**: Search for information

In [ ]:
from langchain_core.tools import tool
from pydantic import BaseModel, Field

# Tool input schemas
class WeatherInput(BaseModel):
    city: str = Field(description="City name to get weather for")

class CalculatorInput(BaseModel):
    expression: str = Field(description="Mathematical expression to evaluate")

class SearchInput(BaseModel):
    query: str = Field(description="Search query")


@tool(args_schema=WeatherInput)
def get_weather(city: str) -> dict:
    """Get the current weather for a city."""
    # Simulated weather data
    weather_data = {
        "new york": {"temperature": 72, "condition": "sunny", "humidity": 45},
        "london": {"temperature": 55, "condition": "cloudy", "humidity": 80},
        "tokyo": {"temperature": 68, "condition": "partly cloudy", "humidity": 60},
        "paris": {"temperature": 61, "condition": "rainy", "humidity": 75},
    }
    city_lower = city.lower()
    if city_lower in weather_data:
        return {"city": city, **weather_data[city_lower]}
    return {"city": city, "error": "City not found", "available_cities": list(weather_data.keys())}


@tool(args_schema=CalculatorInput)
def calculator(expression: str) -> dict:
    """Evaluate a mathematical expression. Supports +, -, *, /, ** operators."""
    try:
        # Safe evaluation of mathematical expressions
        allowed_chars = set("0123456789+-*/.() ")
        if not all(c in allowed_chars for c in expression):
            return {"error": "Invalid characters in expression"}
        
        result = eval(expression)
        return {"expression": expression, "result": result}
    except Exception as e:
        return {"expression": expression, "error": str(e)}


@tool(args_schema=SearchInput)
def search(query: str) -> dict:
    """Search for information on a topic."""
    # Simulated search results
    search_db = {
        "python": [
            {"title": "Python Programming", "snippet": "Python is a high-level programming language known for readability."},
            {"title": "Python Tutorial", "snippet": "Learn Python basics including variables, functions, and classes."},
        ],
        "weather": [
            {"title": "Weather Forecast", "snippet": "Weather is the state of the atmosphere at a given time and place."},
            {"title": "Climate vs Weather", "snippet": "Weather is short-term, climate is long-term average conditions."},
        ],
        "ai": [
            {"title": "Artificial Intelligence", "snippet": "AI is the simulation of human intelligence by machines."},
            {"title": "Machine Learning", "snippet": "ML is a subset of AI that learns from data."},
        ],
    }
    
    query_lower = query.lower()
    for key, results in search_db.items():
        if key in query_lower:
            return {"query": query, "results": results}
    
    return {"query": query, "results": [], "message": "No results found"}


# Collect tools
tools = [get_weather, calculator, search]
print(f"Defined {len(tools)} tools: {[t.name for t in tools]}")

## 2. Create Agent

We'll create a ReAct-style agent using LangChain's tool-calling capabilities.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",  # Use gpt-4 for better tool calling
    temperature=0,
)

# Create prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant with access to various tools.
Use the tools when needed to answer questions accurately.
Always cite the source of your information (e.g., which tool you used)."""),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create agent
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,  # CRITICAL: Needed for CERT tracing
)

print("Agent created successfully!")

## 3. Test Agent Without CERT

First, let's run the agent and see what information is available.

In [ ]:
# Test query
query = "What's the weather in New York? Also, what's 15 * 23?"

# Run agent
start_time = time.time()
result = agent_executor.invoke({"input": query})
duration_ms = (time.time() - start_time) * 1000

print("\n" + "="*60)
print("RESULT STRUCTURE")
print("="*60)
print(f"\nKeys: {result.keys()}")
print(f"\nOutput: {result['output']}")
print(f"\nDuration: {duration_ms:.0f}ms")
print(f"\nIntermediate Steps: {len(result.get('intermediate_steps', []))}")

In [ ]:
# Examine intermediate steps - these contain tool calls!
print("\nINTERMEDIATE STEPS (Tool Calls)")
print("="*60)

for i, (action, observation) in enumerate(result.get("intermediate_steps", [])):
    print(f"\n--- Step {i+1} ---")
    print(f"Tool: {action.tool}")
    print(f"Input: {action.tool_input}")
    print(f"Output: {observation}")

## 4. Method 1: Manual Tracing with CERT

We can manually construct tool calls and trace them. This gives full control but requires more code.

In [ ]:
# Import CERT SDK
import sys
sys.path.insert(0, './cert-sdk-main/src')

from cert import CertClient, extract_context_from_tool_calls

# Initialize client (use mock endpoint for demo)
client = CertClient(
    api_key=CERT_API_KEY,
    dashboard_url="http://localhost:3000",  # Local dev server
)

print("CERT client initialized")
print(f"Auto-extract context: {client.auto_extract_context}")

In [ ]:
def trace_langchain_result_manually(
    client: CertClient,
    query: str,
    result: dict,
    duration_ms: float,
    provider: str = "openai",
    model: str = "gpt-4",
):
    """
    Manually trace a LangChain agent result.
    
    This demonstrates the data extraction process:
    1. Extract tool calls from intermediate_steps
    2. Pass to CERT (context is auto-extracted from tool outputs)
    """
    
    # Extract tool calls from intermediate steps
    tool_calls = []
    intermediate_steps = result.get("intermediate_steps", [])
    
    for action, observation in intermediate_steps:
        tool_call = {
            "name": action.tool,
            "input": action.tool_input if isinstance(action.tool_input, dict) 
                     else {"input": str(action.tool_input)},
            "output": observation if isinstance(observation, (dict, list, str, int, float))
                      else str(observation),
        }
        tool_calls.append(tool_call)
    
    # Show what context will be auto-extracted
    if tool_calls:
        extracted_context = extract_context_from_tool_calls(tool_calls)
        print("\n📋 AUTO-EXTRACTED CONTEXT:")
        print("-" * 40)
        print(extracted_context)
        print("-" * 40)
    
    # Send trace to CERT
    # Note: context is NOT passed - it will be auto-extracted from tool_calls!
    client.trace(
        provider=provider,
        model=model,
        input_text=query,
        output_text=result["output"],
        duration_ms=duration_ms,
        eval_mode="agentic",  # Or "auto" - will detect from tool_calls
        tool_calls=tool_calls,
        goal_description=query,
        metadata={
            "framework": "langchain",
            "tool_count": len(tool_calls),
        },
    )
    
    print(f"\n✅ Traced with {len(tool_calls)} tool calls")
    print(f"   Eval mode: agentic")
    print(f"   Context: auto-extracted ({len(extracted_context) if tool_calls else 0} chars)")
    
    return tool_calls


# Trace the previous result
tool_calls = trace_langchain_result_manually(
    client=client,
    query=query,
    result=result,
    duration_ms=duration_ms,
    model="gpt-4o-mini",
)

## 5. Method 2: Using CERT LangChain Handler (Recommended)

The CERT SDK provides a callback handler that automatically captures everything.

In [ ]:
from cert.integrations.langchain import CERTLangChainHandler

# Create handler
handler = CERTLangChainHandler(
    cert_client=client,
    default_provider="openai",
    default_model="gpt-4o-mini",
    auto_flush=True,
)

print("CERT LangChain handler created!")

In [ ]:
# Run agent with CERT handler
query2 = "Search for information about AI and tell me what you find."

result2 = agent_executor.invoke(
    {"input": query2},
    config={"callbacks": [handler]}  # Add CERT handler!
)

print("\n" + "="*60)
print("RESULT")
print("="*60)
print(f"\n{result2['output']}")

In [ ]:
# Alternative: Use trace_from_result for more control
query3 = "What's 100 divided by 4, plus 50?"

start_time = time.time()
result3 = agent_executor.invoke(
    {"input": query3},
    return_intermediate_steps=True,
)
duration_ms3 = (time.time() - start_time) * 1000

# Trace using helper method
handler.trace_from_result(
    input_text=query3,
    result=result3,
    provider="openai",
    model="gpt-4o-mini",
    duration_ms=duration_ms3,
    metadata={"method": "trace_from_result"},
)

print(f"\n✅ Traced: {query3}")
print(f"   Output: {result3['output']}")

## 6. Compare: With Tools vs Without Tools

Let's demonstrate the difference in evaluation modes when tools are used vs not used.

In [ ]:
# Create a simple chain WITHOUT tools for comparison
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

simple_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions to the best of your knowledge."),
    ("human", "{input}"),
])

simple_chain = simple_prompt | llm | StrOutputParser()

# Ask same question to both
test_query = "What is the capital of France and what's 25 * 4?"

print("="*60)
print("COMPARISON: Agent with Tools vs Simple Chain")
print("="*60)

In [ ]:
# Run with tools (Agentic mode)
print("\n🔧 AGENT WITH TOOLS (Agentic Mode)")
print("-"*40)

start_time = time.time()
agent_result = agent_executor.invoke({"input": test_query})
agent_duration = (time.time() - start_time) * 1000

# Extract tool calls
agent_tool_calls = []
for action, observation in agent_result.get("intermediate_steps", []):
    agent_tool_calls.append({
        "name": action.tool,
        "input": action.tool_input,
        "output": observation,
    })

print(f"Output: {agent_result['output']}")
print(f"Tools used: {[tc['name'] for tc in agent_tool_calls]}")
print(f"Duration: {agent_duration:.0f}ms")

# Trace to CERT
client.trace(
    provider="openai",
    model="gpt-4o-mini",
    input_text=test_query,
    output_text=agent_result["output"],
    duration_ms=agent_duration,
    tool_calls=agent_tool_calls,
    metadata={"comparison": "with_tools"},
)
print("\n✅ Traced as AGENTIC mode (full metrics available)")

In [ ]:
# Run without tools (Generation mode)
print("\n📝 SIMPLE CHAIN WITHOUT TOOLS (Generation Mode)")
print("-"*40)

start_time = time.time()
chain_result = simple_chain.invoke({"input": test_query})
chain_duration = (time.time() - start_time) * 1000

print(f"Output: {chain_result}")
print(f"Tools used: None")
print(f"Duration: {chain_duration:.0f}ms")

# Trace to CERT
client.trace(
    provider="openai",
    model="gpt-4o-mini",
    input_text=test_query,
    output_text=chain_result,
    duration_ms=chain_duration,
    # No tool_calls, no context -> Generation mode
    metadata={"comparison": "without_tools"},
)
print("\n⚠️ Traced as GENERATION mode (limited metrics)")

In [ ]:
# Show the metric difference
print("\n" + "="*60)
print("METRIC AVAILABILITY COMPARISON")
print("="*60)

print("""
┌─────────────────────┬─────────────────┬─────────────────┐
│ Metric              │ Agentic (tools) │ Generation      │
├─────────────────────┼─────────────────┼─────────────────┤
│ Semantic Similarity │ ✅ Available    │ ❌ No context   │
│ NLI Contradiction   │ ✅ Available    │ ❌ No context   │
│ Grounding Score     │ ✅ Available    │ ❌ No context   │
│ SGI (Source-Ground) │ ✅ Available    │ ❌ Undefined    │
│ Tool Integration    │ ✅ Available    │ ❌ No tools     │
├─────────────────────┼─────────────────┼─────────────────┤
│ Instruction Adhere  │ ✅ Available    │ ✅ Available    │
│ Self-Consistency    │ ✅ Available    │ ✅ Available    │
│ Format Compliance   │ ✅ Available    │ ✅ Available    │
└─────────────────────┴─────────────────┴─────────────────┘

Key insight: Without context (from tools or explicit), we cannot
measure FAITHFULNESS or GROUNDING - the core reliability metrics.
""")

## 7. View Stats and Cleanup

In [ ]:
# Get client statistics
stats = client.get_stats()

print("\n📊 CERT CLIENT STATS")
print("="*40)
print(f"Traces sent: {stats['traces_sent']}")
print(f"Traces failed: {stats['traces_failed']}")
print(f"Traces queued: {stats['traces_queued']}")

In [ ]:
# Flush and close
client.flush()
client.close()

print("\n✅ CERT client closed")
print("\nView your traces at: https://dashboard.cert-framework.com")

## Summary

### Key Takeaways

1. **Automatic Context Extraction**: The CERT SDK automatically extracts context from tool outputs in agentic mode. You don't need to construct context manually.

2. **Use `return_intermediate_steps=True`**: When using LangChain agents, always enable this to capture tool calls.

3. **Two Integration Methods**:
   - **Callback Handler** (`CERTLangChainHandler`): Automatic, real-time capture
   - **Manual Tracing**: More control, post-execution tracing

4. **Metric Availability**:
   - With tools/context: Full reliability metrics (SGI, grounding, NLI)
   - Without: Only quality signals (instruction adherence, format)

### Best Practices

```python
# ✅ Good: Let SDK auto-extract context
client.trace(tool_calls=tool_calls, ...)

# ✅ Good: Use callback handler
agent.invoke({"input": query}, config={"callbacks": [handler]})

# ❌ Bad: Agentic mode without tools or context
client.trace(eval_mode="agentic", ...)  # Warning: no context!
```